In [ ]:
#mount drive
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive

# list the contents of /mydrive
!ls /mydrive


In [ ]:
%cd /mydrive/Data/
!pwd
#!unzip "/mydrive/Data/train.zip" -d "/mydrive/Data/train_images"
#!unzip "/mydrive/Data/test.zip" -d "/mydrive/Data/test_images"

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#import PIL
from PIL import Image
import cv2
import pandas as pd
import sklearn
from sklearn import model_selection
from sklearn.metrics import accuracy_score, balanced_accuracy_score , precision_recall_curve, precision_score 
from sklearn.svm import SVC
import skimage
from skimage.io import imread, imshow
import seaborn as sns
from tqdm import tqdm

In [ ]:
path = "/content/gdrive/MyDrive/Data/train_images"
data = pd.read_csv("/content/gdrive/MyDrive/Data/train.txt",sep=" " , names=['Image_Name', 'Label'])
#print(data[0], data[1])
#data = pd.DataFrame(data)
print(data['Label'].value_counts())


In [ ]:

#Labelencoding
from sklearn.preprocessing import LabelEncoder
#from sklearn.preprocessing import OneHotEncoder

# creating instance of labelencoder
labelencoder = LabelEncoder()
#Assigning numerical values and storing in another column
data['Label'] = labelencoder.fit_transform(data['Label'])
print(labelencoder.classes_)
print(data['Label'].value_counts())


# creating instance of one-hot-encoder
'''enc = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame(enc.fit_transform(data[['Label']]).toarray())
data = data.join(enc_df)
data'''

In [ ]:
import os

images = []
labels = []

for x in data['Image_Name']:
  images.append(x)

for y in data['Label']:
  labels.append(y)
#images.append(data[0])
#labels.append(data[1])
print(images,labels)
#print(images[0][0])




In [ ]:
train_path = "/content/gdrive/MyDrive/Data/train_images/train/"
count = 0

plt.figure(figsize=(12,12))

for i in range(10000,10006):
  #print(count)
  #print(image)
  count+=1
  
  train_image_path = os.path.join(train_path,images[i])
  #print(image)
  print(train_image_path)
  image = imread(train_image_path)
  #imshow(image)
  #im = Image.open(train_image_path)
  #width, height = image.size
  #print(width,height)
  shape = image.shape
  print(shape)
  plt.subplot(2,3,count)
  plt.title(labels[i])
  plt.imshow(image)
  #print(image)
  



In [ ]:
#df = pd.DataFrame(data)
data.groupby('Label').size()

In [ ]:
#Data augmentation


In [ ]:
#Feature extraction
from skimage import io
import glob
from skimage.color import rgb2gray
from skimage import feature
from skimage.feature import local_binary_pattern
#import zip

train_image_path = []
for i in range(16929):
  train_image_path.append(os.path.join(train_path,images[i]))
print((train_image_path))

In [ ]:
#Feature extraction
from skimage import io
import glob
from skimage.color import rgb2gray
from skimage import feature
from skimage.feature import local_binary_pattern
#import zip
from skimage.filters import roberts, sobel
from sklearn import preprocessing

class LocalBinaryPatterns:

	def __init__(self, numPoints, radius):
		# store the number of points and radius
		self.numPoints = numPoints
		self.radius = radius

	def describe(self, image, eps=1e-7):
		# compute the Local Binary Pattern representation
		# of the image, and then use the LBP representation
		# to build the histogram of patterns

		lbp = feature.local_binary_pattern(image, self.numPoints,
			self.radius, method="uniform")
		#print(lbp)
		(hist, _) = np.histogram(lbp.ravel(),
			bins=np.arange(0, self.numPoints + 3),
			range=(0, self.numPoints + 2))
	
		# normalize the histogram
		hist = hist.astype("float")
		#hist /= (hist.sum() + eps)
		# return the histogram of Local Binary Patterns
		return lbp,hist

lbp_imgs = []

for i in tqdm(train_image_path[0:16929:1000]):
  #print(train_image_path.index(i))
  count +=1

  image = io.imread(i)
  image = rgb2gray(image)
  
  desc = LocalBinaryPatterns(8, 1)
  hist = desc.describe(image)
  #print(hist)
  lbp_imgs.append(hist)
#print(lbp_imgs)


In [ ]:
#Feature extraction
from skimage import io
import glob
from skimage.color import rgb2gray
from skimage import feature
from skimage.filters.rank import entropy
from skimage.feature import local_binary_pattern
from sklearn import preprocessing
from scipy.stats import skew, kurtosis,norm
from statsmodels import robust

#lbp_imgs = []
image_reshape = []
labels_sub = []
df_lbp= pd.DataFrame()

for i in tqdm(train_image_path[0:16929:1000]):
  #print(train_image_path.index(i))
  count +=1

  df = pd.DataFrame() # temp dataframe

  image = io.imread(i)
  image = rgb2gray(image)
  original_pixel_value = np.array(image.reshape(-1))
  image_reshape.append(original_pixel_value)
  labels_sub.append(labels[train_image_path.index(i)])
  
  desc2 = LocalBinaryPatterns(8, 2)
  lbp2,hist2 = desc2.describe(image) #lbp extraction
  '''desc3 = LocalBinaryPatterns(8, 3)
  lbp3,hist3 = desc3.describe(image)
  desc5 = LocalBinaryPatterns(8,5)
  lbp5,hist5 = desc5.describe(image)
  desc9 = LocalBinaryPatterns(8,9)
  lbp9,hist9 = desc9.describe(image)'''

  lbp2_mean =[]
  lbp2_std = []
  lbp2_median = []
  lbp2_max = []
  lbp2_min = []
  lbp2_skew = []
  lbp2_kurtosis = []
  lbp2_mad = []
  lbp2_range = []

  lbp2_mean.append(np.mean(lbp2))  #appending to dataframe
  df["lbp2_mean"] = lbp2_mean
  #print(lbp2_mean)

  lbp2_std.append(np.std(lbp2))
  df["lbp2_std"] = lbp2_std

  lbp2_median.append(np.median(lbp2))
  df["lbp2_median"] = lbp2_median

  lbp2_max.append(np.max(lbp2))
  df["lbp2_max"] = lbp2_max

  lbp2_skew.append(skew(hist2))
  df["lbp2_skew"] = lbp2_std

  lbp2_kurtosis.append(kurtosis(hist2))
  df["lbp2_kurtosis"] = lbp2_kurtosis

  lbp2_range.append(np.ptp(lbp2))
  df["lbp2_range"] = lbp2_range


  df_lbp = df_lbp.append(df,ignore_index=True) #append to fial dataframe that has to be given to classifier

print(df_lbp)



In [ ]:
df_final = pd.DataFrame()
df_final = df_final.append(df_lbp)
df_final['labels'] = labels_sub
#print(df_final)


In [ ]:
#X= np.asarray(df_final[:-1], dtype = object)
X= df_final.iloc[: , :-1]
#print(X)
y = df_final['labels']
#y = np.asarray(df_final['labels'])
#print(x)
#print(y)

In [ ]:
from sklearn.model_selection import KFold

kf5 = KFold(n_splits=5, shuffle=True)
for train_index, test_index in kf5.split(X):
    print(train_index, test_index)
    


In [ ]:
# divide train and validation
from sklearn.model_selection import train_test_split

X_train , x_test , y_train, y_test = train_test_split(df_lbp,y, test_size = 0.2 , random_state = 8)
#y_train = y_train.resize((1, 1))
#print(X_train)

In [ ]:
#svm modelling

from sklearn.svm import SVC

classifier = SVC(kernel = 'poly', C=2, random_state = 2)
classifier.fit(X_train,y_train, sample_weight=None)

y_predict = classifier.predict(x_test)


In [ ]:
#Evaluation

from sklearn.metrics import balanced_accuracy_score,classification_report

#print(classification_report(y_test,y_predict))
print(balanced_accuracy_score)
print( 'y_test:  ' , y_test)
print( 'y_predict:' ,y_predict)
balanced_accuracy_score(y_test, y_predict)